In [ ]:
%matplotlib notebook
import numpy as np
import cmath
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

## Evanescence

For $\theta_i > \theta_{crit}$ we can write

$$\cos{\theta_t} = (1 - \sin^2{\theta_t})^{1/2} = (1 - (\eta_1 / \eta_2)^2 \sin^2{\theta_i})^{1/2} = ib$$

Then

$$r_s = \frac{(\eta_1 / \eta_2) \cos{\theta_i} - \cos{\theta_t}}{(\eta_1 / \eta_2) \cos{\theta_i} + \cos{\theta_t}} = \frac{a_s - ib}{a_s + ib}$$

where $a_s = (\eta_1 / \eta_2) \cos{\theta_i}$. Similarly,

$$r_p = -\frac{(\eta_2 / \eta_1) \cos{\theta_i} - \cos{\theta_t}}{(\eta_2 / \eta_1) \cos{\theta_i} + \cos{\theta_t}} = -\frac{a_p - ib}{a_p + ib}$$

where $a_p = (\eta_2 / \eta_1) \cos{\theta_i}$.

Both have $\left|r\right| = \frac{\left|a - ib\right|}{\left|a + ib\right|} = 1$

The transmitted wave is

$$E_t = E_{t0} e^{i (\mathbf{k}_t \cdot \mathbf{r} - \omega t)} = E_{t0} e^{i (k_t \sin{\theta_t} x + k_t \cos{\theta_t} y - \omega t)}$$

Using $k_t = \frac{\eta_2 \omega}{c}$,

$$k_t \sin{\theta_t} = \frac{\eta_2 \omega}{c} \frac{\eta_1}{\eta_2} \sin{\theta_i} = \frac{\eta_1 \omega}{c} \sin{\theta_i} = k_i \sin{\theta_i}$$

$$\Rightarrow E_t = E_{t0} e^{i \left(k_i \sin{\theta_i} x + i k_t b y - \omega t\right)}
= E_{t0} e^{i \left(k_i \sin{\theta_i} x - \omega t\right)} e^{-k_t b y}$$

This solution is a surface wave that decays away from the interface with a scale distance of $\frac{1}{k_t b} = \frac{c}{\eta_2 \omega b}$

In [ ]:
class Boundary:
    def __init__(self, angle, n1, n2, polarisation, interference=True):
        """
        Creates a list of frames that progress in time

        Args:
            angle (float) - [degs] {0 to 90}
            # out (bool) - True if outgoing, False if incoming (to the origin)

            # E_0 (float) - Magnitude of the E field
            polarisation (str) - {'s' or 'p'} Whether E or B is parallel to the boundary
            # w (float) - [rad s^-1] Angular frequency (default: green light)
            n1 (float) - The incident material's refractive index
            n2 (float) - The second material's refractive index

            interference
        """
        self.theta = np.deg2rad(angle)
#         self.theta_crit = self.find_critical_angle()
        self.n1 = n1
        self.n2 = n2
        if polarisation == "s" or polarisation == "p":
            self.polarisation = polarisation
        else:
            raise Exception('Polarisation argument must be "s" or "p"')
        
        self.frame = 0
        self.num_frames = 100
        self.graph_dim = 100
        
        self.incident = self.create_wave(theta=self.theta, amplitude=1, material=1)
        self.transmitted = self.transmit()
        if interference and self.n1 != self.n2:
            self.reflected = self.reflect()
            self.incident["zz"] += self.reflected["zz"]
        

    def create_wave(self, theta, amplitude, material, reverse_phase=False):
        """
        Args:
            theta (float) - [rads] {0 to π/2}
            amplitude
            material (int) - {1 or 2} Whether the wave is in material 1 or 2
            reverse_phase (bool)
        """        
        if material == 1:
            x = np.linspace(-1, 0, self.graph_dim/2)
            n = self.n1
        elif material == 2:
            x = np.linspace(0, 1, self.graph_dim/2)
            n = self.n2
        else:
            raise ValueError("arg 'material' must be 1 or 2")
        y = np.linspace(-1, 1, self.graph_dim)
            
        xx, yy = np.meshgrid(x, y)
        
        k_x = np.cos(theta) * n
        k_y = -np.sin(theta) * n
        
        if reverse_phase == False:
            zz = np.array([amplitude * np.exp(8j*np.pi * (k_x*xx + k_y*yy - phase)).real for phase in np.linspace(0, 0.25, self.num_frames)])
        elif reverse_phase == True:
            zz = np.array([amplitude * np.exp(8j*np.pi * (k_x*xx + k_y*yy + phase)).real for phase in np.linspace(0, 0.25, self.num_frames)])
        else:
            raise TypeError("arg `reverse_phase` must be of type('bool')")
            
        return {"x": x, "y": y, "zz": zz}
        
    
    def transmit(self):
        cos_theta_i = np.cos(self.theta)
        cos_theta_t = self.cos_snell(self.n1, self.n2, self.theta)
        plot_theta_t = cmath.acos(cos_theta_t)
        if isinstance(cos_theta_t, complex):
            print('Total internal reflection')
            cos_theta_t = 0
        
        if self.polarisation == "s":
            t = (2. * self.n1 * cos_theta_i) / (self.n1 * cos_theta_i + self.n2 * cos_theta_t)
        else:
            t = (2. * self.n1 * cos_theta_i) / (self.n1 * cos_theta_t + self.n2 * cos_theta_i)
                
        return self.create_wave(theta=plot_theta_t, amplitude=t, material=2)
    
    
    def reflect(self):
        if self.n1 == self.n2:
            print('Refractive indices equal - no reflection')
            return None
        
        cos_theta_i = np.cos(self.theta)
        theta_r = self.theta
        cos_theta_t = self.cos_snell(self.n1, self.n2, self.theta)
        if isinstance(cos_theta_t, complex):
            cos_theta_t = 0
        
        plot_theta_r = -theta_r
        
        if self.polarisation == "s":
            r = (self.n1 * cos_theta_i - self.n2 * cos_theta_t) / (self.n1 * cos_theta_i + self.n2 * cos_theta_t)
        else:
            r = (self.n1 * cos_theta_t - self.n2 * cos_theta_i) / (self.n1 * cos_theta_t + self.n2 * cos_theta_i)
        
        return self.create_wave(theta=plot_theta_r, amplitude=r, material=1, reverse_phase=True)
        
        
#     def find_critical_angle(self):
#         if self.n1 > self.n2:
#             return 0.5*np.pi - np.arcsin(n2/n1)
#         else:
#             return 0.5*np.pi + 0.0001
    
    
    @staticmethod
    def cos_snell(n1, n2, theta_i):
        """
        Returns cosine of angle of transmission using Snell's law
        
        Args:
            n1 (float) - incident medium's refractive index
            n2 (float) - transmissive medium's refractive index
            theta_i (float) - angle of incidence
        """
        cos_squared_theta_t = 1. - ((n1/n2) * np.sin(theta_i))**2
        if cos_squared_theta_t < 0:
            return cmath.sqrt(cos_squared_theta_t)
        else:
            return np.sqrt(cos_squared_theta_t)
        
    
    def plot(self):
        fig = plt.figure(figsize=(5, 5))

        im_i = plt.pcolormesh(self.incident["x"], self.incident["y"], self.incident["zz"][0], vmin=-1, vmax=1)#, animated=True)
        im_t = plt.pcolormesh(self.transmitted["x"], self.transmitted["y"], self.transmitted["zz"][0], vmin=-1, vmax=1)#, animated=True)
    
        def update_plot(*args):
            im_i.set_array(self.incident["zz"][self.frame][:-1, :-1].ravel())
            im_t.set_array(self.transmitted["zz"][self.frame][:-1, :-1].ravel())
            
            self.frame += 1
            if self.frame == self.num_frames:
                self.frame = 0

            return im_i, im_t
        
        ani = animation.FuncAnimation(fig, update_plot, interval=50, blit=True)
        
        plt.show()
        return ani
    

In [ ]:
boundary = Boundary(angle=53.6, n1=1., n2=1.5, polarisation="s", interference=True)

ani = boundary.plot()

In [ ]:
def run(angle, n1, n2, frames, interferance=True, graph=True):
    theta_i_deg = angle

    # calculate critical angle if appropriate
    theta_crit = 0.5*np.pi + 0.0001
    if n1>n2:
        theta_crit - np.arcsin(n2/n1)

    theta_i = np.deg2rad(theta_i_deg)

    #Time normalised in wave periods
    omega = 2.0*np.pi

    #Distances normalised by vacuum wavelength of wave
    k1 = 2.0*np.pi*n1
    k2 = 2.0*np.pi*n2

    #Use Snells Law
    cos_theta_t = cmath.sqrt(1.0-(n1*np.sin(theta_i)/n2)**2)

    #Calculate normalised components of k vectors
    #x direction is parallel to interfeace
    #y direction is normal to interface
    k1x = k1*np.sin(theta_i)
    k1y = k1*np.cos(theta_i)
    k2x = k1x
    k2y = k2*cos_theta_t

    #Calculate reflection anf transmission coefficients
    if theta_i < theta_crit:
        denom = n1*cos_theta_t + n2*np.cos(theta_i)
        rp = (n1*cos_theta_t - n2*np.cos(theta_i))/denom
        tp = 2.0*n1*np.cos(theta_i)/denom
        energyCheck = rp**2 + tp**2*n2*cos_theta_t/(n1*np.cos(theta_i))
    else:
        rp = 1.
        tp = 2.0*n1/n2
        energyCheck=rp**2

    #Maximum Amplitude
    amp = [1.0, abs(rp) + 1, abs(tp)]
    maxAmp = max(amp)

    t_0 = 0
    tFinal = 3
    numFrames = frames
    interval = (tFinal - t_0)/numFrames

    xMax = 2.0
    yMin = -2.0
    yMax = 2.0
    x = np.arange(0., xMax, 0.02)
    y = np.arange(yMin, yMax, 0.02)
    X,Y = np.meshgrid(x, y)

    logindpos = Y>=0
    logindneg = Y<0
    
    data =[]
    raw=[]
    for k in range(0,numFrames):
        t = t_0 + k*interval

        E1inc = (np.exp(1j*(k1x*X + k1y*Y - omega*t))).real
        E1tot = (np.exp(1j*(k1x*X + k1y*Y - omega*t))).real + (rp*np.exp(1j*(k1x*X - k1y*Y - omega*t))).real
        Etrans = (tp*np.exp(1j*(k2x*X + k2y*Y - omega*t))).real
        
        if interferance==True:
            E1 = np.multiply(logindneg, E1tot)+ np.multiply(logindpos, Etrans)
        else:
            E1 = np.multiply(logindneg, E1inc)+ np.multiply(logindpos, Etrans)

        #trace = go.Heatmap(z=E1, showscale=False)
        trace = go.Contour(z=E1, colorscale='Viridis', showscale=False, contours = dict(coloring = 'heatmap', showlines=False))
        trace['z'] = trace['z'].tolist()
        data.append(trace)
        E1 = E1.tolist()
        raw.append(E1)
    
    if graph:
         return data
    else:
        return raw
   

In [ ]:
data = run(angle=45, n1=1., n2=1.5, frames=10, interferance=True, graph=True)
layout = dict(width=500, height=500)

In [ ]:
fig=dict(data=[data[0]], layout=layout)
iplot(fig)

In [ ]:
# import json

# with open('heatmaps.JSON', 'w') as outfile: 
#     json.dump(data, outfile)

In [ ]:
# raw = run(angle=40, n1=1.5, n2=1., frames=10, interferance=False, graph=False)

In [ ]:
# import json

# with open('rawHeatData.JSON', 'w') as outfile: 
#     json.dump(raw, outfile)